In [1]:
import pandas as pd
import geopandas as gpd
import random
import shutil
import zipfile
import matplotlib.pyplot as plt
from pathlib import Path
import random, os, sys
import numpy as np
import zipfile
import seaborn as sns

sys.path.insert(0, 'C:/Users/mmh/Documents/Codes/cross-sectro-transp-energy-model/')
# importlib.reload(sys.modules['src.calculation'])
from src.func_buildings import *
pd.set_option('display.max_columns', None)
data_dir = 'C:/Users/mmh/Documents/Data/'
data_maps = 'C:/Users/mmh/OneDrive - Oak Ridge National Laboratory/Melrose/8.Maps/'

# GA, Fulton County

## Building locations

In [2]:
building_loc = pd.read_csv(data_dir+'Buildings/GA/Fulton.csv', index_col=False)
print(len(building_loc))
building_loc[['latitude','longitude']] = building_loc.Centroid.str.split("/", expand = True)
building_loc = gpd.GeoDataFrame(building_loc, geometry=gpd.points_from_xy(building_loc.longitude, building_loc.latitude), 
                                crs='EPSG:4269')    
building_loc.head(3)

247757


,ID,Centroid,Footprint2D,State,Area,Area2D,Height_m,NumFloors,WWR_surfaces,CZ,BuildingType,Standard,lat,lon,.points,index_right,City,latitude,longitude,geometry
0,5007013188280,33.6078469296821/-84.58397037226572,33.607634/-84.584013_33.607697/-84.584096_33.6...,GA,19334.692351,9667.346175,11.300995,2.0,.07_.07_.07_.07_.07_.07_.07_.07_.07_.07_.07_.0...,3A,RetailStandalone,90.1-2007,33.607847,-84.583970,POINT (-84.58397037226572 33.6078469296821),0,Fulton,33.6078469296821,-84.58397037226572,POINT (-84.58397 33.60785)
1,5007013188281,33.61078985911062/-84.47664704780213,33.610636/-84.477217_33.610957/-84.477212_33.6...,GA,158542.947727,39635.736932,15.035895,4.0,.15_.15_.15_.15_.15_.15_.15_.15_.15_.15_.15_.15,3A,Hospital,DOE-Ref-1980-2004,33.610790,-84.476647,POINT (-84.47664704780213 33.61078985911062),0,Fulton,33.61078985911062,-84.47664704780213,POINT (-84.47665 33.61079)
2,5007013188282,33.57899273577419/-84.7689914359184,33.579322/-84.768879_33.579025/-84.768875_33.5...,GA,63766.728573,15941.682143,15.293895,4.0,.33_.33_.33_.33_.33_.33_.33_.33_.33_.33_.33_.3...,3A,MediumOffice,DOE-Ref-1980-2004,33.578993,-84.768991,POINT (-84.76899143591839 33.57899273577419),0,Fulton,33.57899273577419,-84.7689914359184,POINT (-84.76899 33.57899)


In [3]:
building_loc.BuildingType.value_counts()

MidriseApartment          58840
MediumOffice              54928
HighriseApartment         41090
IECC                      38111
LargeOffice               28097
SmallHotel                19570
SmallOffice                1570
RetailStandalone           1370
LargeHotel                 1079
Hospital                    784
RetailStripmall             625
Warehouse                   488
Outpatient                  344
SecondarySchool             336
QuickServiceRestaurant      196
FullServiceRestaurant       171
PrimarySchool               158
Name: BuildingType, dtype: int64

In [4]:
GA_shp = gpd.read_file(data_maps+'geo/tl_2022/tl_2022_13_bg.zip', index_col=False)
print(len(GA_shp[GA_shp.COUNTYFP=='121'])) # number of block groups
GA_shp.head(3)

858


,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,13,217,100202,2,132171002022,Block Group 2,G5030,S,21731901,29568,+33.5859573,-083.7029124,"POLYGON ((-83.74737 33.60075, -83.74681 33.600..."
1,13,089,022007,1,130890220071,Block Group 1,G5030,S,508385,0,+33.8137467,-084.2481327,"POLYGON ((-84.25174 33.81598, -84.25086 33.816..."
2,13,089,022007,2,130890220072,Block Group 2,G5030,S,705047,0,+33.8078304,-084.2456120,"POLYGON ((-84.25074 33.80853, -84.25065 33.808..."


In [5]:
building_loc = building_loc.sjoin(GA_shp[['STATEFP','COUNTYFP','TRACTCE','BLKGRPCE','GEOID','geometry']], how="left", predicate="within"
                                  , lsuffix='_left', rsuffix='_right')

building_loc['BuildingClass'] = 'Commercial'
building_loc.loc[building_loc.BuildingType.isin(['MidriseApartment','HighriseApartment','IECC']),'BuildingClass'] = 'Residential'
building_loc.loc[building_loc.BuildingType.isin(['SmallOffice','MediumOffice','LargeOffice',
                                              'PrimarySchool','SecondarySchool']),'BuildingClass'] = 'Industrial'

%store building_loc
# building_loc.to_file('GA_buildings.shp', driver='ESRI Shapefile')
building_loc.head()

Stored 'building_loc' (GeoDataFrame)


,ID,Centroid,Footprint2D,State,Area,Area2D,Height_m,NumFloors,WWR_surfaces,CZ,BuildingType,Standard,lat,lon,.points,index_right,City,latitude,longitude,geometry,index__right,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,BuildingClass
0,5007013188280,33.6078469296821/-84.58397037226572,33.607634/-84.584013_33.607697/-84.584096_33.6...,GA,19334.692351,9667.346175,11.300995,2.0,.07_.07_.07_.07_.07_.07_.07_.07_.07_.07_.07_.0...,3A,RetailStandalone,90.1-2007,33.607847,-84.583970,POINT (-84.58397037226572 33.6078469296821),0,Fulton,33.6078469296821,-84.58397037226572,POINT (-84.58397 33.60785),587,13,121,010526,2,131210105262,Commercial
1,5007013188281,33.61078985911062/-84.47664704780213,33.610636/-84.477217_33.610957/-84.477212_33.6...,GA,158542.947727,39635.736932,15.035895,4.0,.15_.15_.15_.15_.15_.15_.15_.15_.15_.15_.15_.15,3A,Hospital,DOE-Ref-1980-2004,33.610790,-84.476647,POINT (-84.47664704780213 33.61078985911062),0,Fulton,33.61078985911062,-84.47664704780213,POINT (-84.47665 33.61079),668,13,121,010522,1,131210105221,Commercial
2,5007013188282,33.57899273577419/-84.7689914359184,33.579322/-84.768879_33.579025/-84.768875_33.5...,GA,63766.728573,15941.682143,15.293895,4.0,.33_.33_.33_.33_.33_.33_.33_.33_.33_.33_.33_.3...,3A,MediumOffice,DOE-Ref-1980-2004,33.578993,-84.768991,POINT (-84.76899143591839 33.57899273577419),0,Fulton,33.57899273577419,-84.7689914359184,POINT (-84.76899 33.57899),560,13,121,010402,3,131210104023,Industrial
3,5007013188283,33.62292022283877/-84.50243209502493,33.622629/-84.502555_33.622628/-84.502804_33.6...,GA,131502.094459,18786.013494,24.206707,7.0,.27_.27_.27_.27_.27_.27_.27_.27_.27,3A,LargeHotel,DOE-Ref-1980-2004,33.622920,-84.502432,POINT (-84.50243209502493 33.62292022283877),0,Fulton,33.62292022283877,-84.50243209502493,POINT (-84.50243 33.62292),3529,13,121,011306,1,131210113061,Commercial
4,5007013188284,33.570989326293905/-84.54456733157187,33.570862/-84.54485_33.571217/-84.544843_33.57...,GA,119183.107776,23836.621555,18.658968,5.0,.27_.27_.27_.27_.27_.27_.27,3A,LargeHotel,DOE-Ref-1980-2004,33.570989,-84.544567,POINT (-84.54456733157187 33.5709893262939),0,Fulton,33.570989326293905,-84.54456733157187,POINT (-84.54457 33.57099),559,13,121,010524,1,131210105241,Commercial


# Selected Block Groups

In [6]:
# selected_ID = gpd.read_file(data_dir+"Buildings/GA/Atlanta downtown.shp")
# selected_ID = gpd.read_file(data_dir+"Buildings/GA/Fulton rural.shp")
selected_ID = building_loc.copy()
selected_ID = selected_ID.ID.astype(str).values
print(len(selected_ID))
selected_ID[0]

247757


'5007013188280'

# Unzip files

In [7]:
zip_file_path = data_dir + 'Buildings/Fulton_IDF.zip'

for i in selected_ID:
    # Specify the file to be extracted (including the subfolder)
    file_to_extract = 'ModelsOrig/5009016211541/5009016211541.idf'
    file_to_extract = 'ModelsOrig/'+str(i)+'/'+str(i)+'.idf'

    # Specify the path to the new folder where the file will be extracted
    extracted_folder_path = data_dir + 'Buildings/IDF_Fulton_Orig_2'

    # Open the zip file
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        # Extract the specified file to a temporary folder
        temp_folder = os.path.join(extracted_folder_path, 'temp')
        zip_ref.extract(file_to_extract, temp_folder)

    # Move the extracted file to the desired location without subfolders
    extracted_file_path = os.path.join(temp_folder, file_to_extract)
    destination_path = os.path.join(extracted_folder_path, os.path.basename(file_to_extract))
    os.rename(extracted_file_path, destination_path)

    # Remove the temporary folder
    shutil.rmtree(temp_folder, ignore_errors=True)
    print('Finish '+i)

Finish 5007013188282
Finish 5007013188301
Finish 5007013188306
Finish 5007013188323
Finish 5007013188409
Finish 5007013188411
Finish 5007013188412
Finish 5007013188448
Finish 5007013188459
Finish 5007013188468
Finish 5007013188471
Finish 5007013188474
Finish 5007013188493
Finish 5007013188522
Finish 5007013188525
Finish 5007013188580
Finish 5007013188596
Finish 5007013188597
Finish 5007013188620
Finish 5007013188696
Finish 5007013188802
Finish 5007013188816
Finish 5007013188849
Finish 5007013188870
Finish 5007013188881
Finish 5007013188902
Finish 5007013188922
Finish 5007013188928
Finish 5007013188977
Finish 5007013189013
Finish 5007013189158
Finish 5007013189252
Finish 5007013189260
Finish 5007013189266
Finish 5007013189306
Finish 5007013189384
Finish 5007013189407
Finish 5007013189414
Finish 5007013189425
Finish 5007013189437
Finish 5007013189476
Finish 5007013189525
Finish 5007013189538
Finish 5007013189545
Finish 5007013189592
Finish 5007013189612
Finish 5007013189631
Finish 500701

# Building capacity

In [8]:
# loc = data_dir+'Buildings/IDF_Fulton/ModelsOrig/'
loc = data_dir + 'Buildings/IDF_Fulton_Orig_2/'
name_subset = ['People per Zone Floor Area {person/m2}']
# filenames = random.sample(os.listdir(loc), 5)
bname = selected_ID
# filenames = bname + '/'+bname+'.idf'
filenames = bname+'.idf'
building_set = process_idf(loc,filenames,type_subset=[],name_subset=name_subset,skip=True)
print(len(set(building_set.ID)))
building_set.head(2)

finish processing building 5007013188282.idf
finish processing building 5007013188301.idf
finish processing building 5007013188306.idf
finish processing building 5007013188323.idf
finish processing building 5007013188409.idf
finish processing building 5007013188411.idf
finish processing building 5007013188412.idf
finish processing building 5007013188448.idf
finish processing building 5007013188459.idf
finish processing building 5007013188468.idf
finish processing building 5007013188471.idf
finish processing building 5007013188474.idf
finish processing building 5007013188493.idf
finish processing building 5007013188522.idf
finish processing building 5007013188525.idf
finish processing building 5007013188580.idf
finish processing building 5007013188596.idf
finish processing building 5007013188597.idf
finish processing building 5007013188620.idf
finish processing building 5007013188696.idf
finish processing building 5007013188802.idf
finish processing building 5007013188816.idf
finish pro

,0,parameter,name,value,label,ID
5916,0.096347515 !Peop...,0.096347515,People per Zone Floor Area {person/m2},0.096347515,None,5007013188282
12653,0.028239119 !Peop...,0.028239119,People per Zone Floor Area {person/m2},0.028239119,None,5007013188301


In [9]:
building_set['ID'] = building_set['ID'].str.split("/", expand=True)[0].astype('int64')
building_set = pd.merge(building_set,building_loc[['ID','GEOID']],how='left',on='ID')
building_set.head()

,0,parameter,name,value,label,ID,GEOID
0,0.096347515 !Peop...,0.096347515,People per Zone Floor Area {person/m2},0.096347515,None,5007013188282,131210104023
1,0.028239119 !Peop...,0.028239119,People per Zone Floor Area {person/m2},0.028239119,None,5007013188301,131210103091
2,0.028075866 !Peop...,0.028075866,People per Zone Floor Area {person/m2},0.028075866,None,5007013188323,131210103091
3,0.028075866 !Peop...,0.028075866,People per Zone Floor Area {person/m2},0.028075866,None,5007013188409,131210103091
4,0.096347515 !Peop...,0.096347515,People per Zone Floor Area {person/m2},0.096347515,None,5007013188411,131210105402


In [10]:
# examine: some buildings do not have this information - may not need to simulate?
inclusion_ID = building_set[building_set.name=='People per Zone Floor Area {person/m2}'].ID 
len(inclusion_ID)

3207

In [11]:
### need to double check whether this parameter is used
building_cap = building_set[building_set.name=='People per Zone Floor Area {person/m2}'] # some buildings do not have this information - may not need to simulate?
# building_cap[building_cap.isna().any(axis=1)]

selected_cols = ['ID','GEOID','Area','latitude','longitude']
building_cap = pd.merge(building_cap,building_loc[selected_cols],how='left',on=['ID','GEOID'])

building_cap['capacity'] = building_cap['value'].astype(float) * building_cap['Area'] * 0.092903
building_cap = building_cap[selected_cols+['capacity']]
print(len(set(building_cap.ID)))

%store building_cap

building_cap.head()

3207
Stored 'building_cap' (DataFrame)


,ID,GEOID,Area,latitude,longitude,capacity
0,5007013188282,131210104023,63766.728573,33.57899273577419,-84.7689914359184,570.774278
1,5007013188301,131210103091,216240.822823,33.63734722015903,-84.7159930413506,567.307555
2,5007013188323,131210103091,81248.953328,33.63490535003784,-84.70748775605293,211.924259
3,5007013188409,131210103091,4860.119042,33.627810000000004,-84.722174,12.676805
4,5007013188411,131210105402,963.712308,33.53337218757921,-84.55780135614702,8.626163


In [12]:
building_cap.to_feather(data_dir+'Buildings/GA/building_cap_rural.feather')
# building_cap.to_feather(data_dir+'Buildings/GA/building_cap_100BG.feather')

In [13]:
building_cap[building_cap.isna().any(axis=1)]

,ID,GEOID,Area,latitude,longitude,capacity


# Building capacity - ALL

In [22]:
df = pd.read_feather(data_dir+'Buildings/GA/building_cap_rural.feather')
df = pd.merge(df,building_loc[['ID','GEOID','BuildingType']],how='left',on=['ID','GEOID'])
df['perArea'] = df['capacity']/(df['Area'])
df = df.groupby('BuildingType')['perArea'].agg([('meanPerArea', 'mean'), ('std', 'std')]).reset_index()
building_cap = pd.merge(building_loc[['ID','GEOID','Area','latitude','longitude','BuildingType']],df[['BuildingType','meanPerArea']],on='BuildingType',how='left')
building_cap['capacity'] = building_cap['Area']*building_cap['meanPerArea']
building_cap = building_cap.drop(columns=['BuildingType','meanPerArea'])
print(len(building_cap))
building_cap.head()

247757


,ID,GEOID,Area,latitude,longitude,capacity
0,5007013188280,131210105262,19334.692351,33.6078469296821,-84.58397037226572,252.654461
1,5007013188281,131210105221,158542.947727,33.61078985911062,-84.47664704780213,430.113149
2,5007013188282,131210104023,63766.728573,33.57899273577419,-84.7689914359184,570.774278
3,5007013188283,131210113061,131502.094459,33.62292022283877,-84.50243209502493,2471.108442
4,5007013188284,131210105241,119183.107776,33.570989326293905,-84.54456733157187,2239.617437


In [23]:
building_cap.to_feather(data_dir+'Buildings/GA/building_cap_all.feather')

# Building types from IDF files

In [14]:
# loc = data_dir+'Buildings/IDF_AZ/AZ_Pima'
# type_subset = ['Building']
# name_subset = ['Name']
# filenames = building_loc.ID.astype(str)+'.idf'
# buildingType = process_idf(loc,filenames,type_subset,name_subset)

# buildingType['BuildingType_idf'] = buildingType['value'].str.split('-').str[1]
# buildingType.BuildingType_idf.value_counts()

In [15]:
# type_comparison = pd.merge(building_loc,buildingType[['ID','BuildingType_idf']],how='left',on=['ID'])
# type_comparison = type_comparison[['ID','BuildingType','BuildingType_idf']]
# type_comparison = type_comparison[(type_comparison.BuildingType!='IECC') & (type_comparison.BuildingType!=type_comparison.BuildingType_idf)]
# type_comparison.to_csv(data_dir+'Buildings/'+'building_types.csv',index=False)
# type_comparison.head()

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\Users\mmh\Anaconda3\envs\py39sim\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Users\mmh\Anaconda3\envs\py39sim\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\Users\mmh\Anaconda3\envs\py39sim\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Users\mmh\Anaconda3\envs\py39sim\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] 